In [51]:
import numpy as np
import pandas as pd

import plotly
import plotly.express as px

filenames = ['error1_10-40_120s.csv', 'error2_10-40_120s.csv', 'error3_10-40_120s.csv',
             'error4_10-40_120s.csv', 'error5_10-40_120s.csv', 'error6_10-40_120s.csv',
             'error7_10-40_120s.csv', 'error8_10-40_120s.csv', 'error9_10-40_120s.csv', 
             'error10_10-40_120s.csv']

In [52]:
data = []

for filename in filenames:
    file = pd.read_csv(filename)
    data.append(file[1:])

In [53]:
max_displacement_raw = 0.0058162267839687
min_displacement_raw = 0.0320625610948191

min_displacement_mm = 0
max_displacement_mm = 1

def conv_displacement_to_mm(x):
    return (x - min_displacement_raw) * (max_displacement_mm - min_displacement_mm) / (max_displacement_raw - min_displacement_raw) + min_displacement_mm

In [54]:
for experiment in data:
    for i, value in enumerate(experiment['y_val']):
        experiment['y_val'][i] = conv_displacement_to_mm(value)

In [55]:
plot_data = []

for i, experiment in enumerate(data):
    plot_data.append([])
    for j in range(0, len(experiment), 100):
        try:
            plot_data[i].append(
                {
                    'frequency': experiment['frequency'][j], 
                    'displacement_y': np.mean(experiment[j:j+100]['y_val'])
                }
            )
        except Exception:
            pass
    
    plot_data[i] = pd.DataFrame(plot_data[i])


In [58]:
diff_sum = 0

for i in range(len(plot_data) - 1):
    diff_sum += np.mean(np.abs(plot_data[i]['displacement_y'] - plot_data[i+1]['displacement_y']))

displacement_err = diff_sum / 9
displacement_err

0.019410482226018842

In [57]:
f0 = []

for i, experiment in enumerate(plot_data):
    min_val = experiment['displacement_y'].idxmax()
    f0.append(experiment['frequency'][min_val])
    
err = max(f0) - min(f0)
err

0.5322978496551549

In [48]:
f0

[28.399029473463692,
 28.414716124534607,
 28.40242862701416,
 28.223326702912647,
 28.18163573741913,
 28.177601397037503,
 28.355974356333416,
 28.356605172157288,
 28.401906589667004]

In [40]:
for i in range(len(plot_data)):
    fig = px.line(plot_data[i], x=plot_data[i]['frequency'], y=plot_data[i]['displacement_y'], title='mopslik')
    fig.show()